In [28]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import r2_score
from tensorflow.keras import utils,losses,layers,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dot, Softmax, Concatenate,Conv1D,Dropout,MaxPooling1D, Flatten
from tensorflow.keras.models import Model
import pydot
from keras.utils.vis_utils import model_to_dot

In [29]:
data_path="winter.csv"
dataset=pd.read_csv(data_path,parse_dates=['Date'],index_col=['Date'])

In [30]:
future=['NH3']
for col in future:
    scaler=MinMaxScaler()
    if(col not in ['sin(h)','cos(h)']):
    	dataset[col]=scaler.fit_transform(dataset[col].values.reshape(-1,1))
dataset

,Indoor_T,Indoor_RH,Outdoor_T,Outdoor_RH,NH3,TSP,CO2,TVOC,PM10,PM2.5,Wind_force,Wind_speed
Date,,,,,,,,,,,,
2023-02-08 17:20:00,0.305772,78.053284,0.283425,80.555725,3.7,7,822,0.015088,196,167,0.000000,0.000000
2023-02-08 17:30:00,0.311232,78.486633,0.276915,81.059265,4.0,6,816,0.018972,186,151,0.333333,0.025586
2023-02-08 17:40:00,0.301872,79.411316,0.272909,81.752014,3.7,7,818,0.023603,189,158,0.333333,0.021322
2023-02-08 17:50:00,0.316693,80.488586,0.267902,82.125854,3.3,1335,884,0.161637,207,165,0.000000,0.000000
2023-02-08 18:00:00,0.265991,88.589478,0.262894,82.583618,3.5,824,1070,0.373917,182,158,0.666667,0.027719
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-09 12:30:00,0.729329,61.466980,0.798698,54.022217,0.1,89,521,0.028832,74,54,0.666667,0.733475
2023-03-09 12:40:00,0.721529,61.929321,0.812719,52.728271,0.1,78,525,0.029429,64,51,0.666667,0.722814
2023-03-09 12:50:00,0.727769,61.595154,0.813220,53.625488,0.1,89,531,0.031969,74,54,0.666667,0.716418


In [31]:
def multivariate_data(x,y, start_index, end_index, history_size,
                      target_size, step, single_step):
    data = []
    labels = []

    start_index = start_index + history_size
    
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step) 
        mid_data=x.iloc[indices]
        data.append(mid_data)

        if single_step:
            mid_data=y.iloc[i+target_size]
            labels.append(mid_data)
        else:
            labels.append(y.iloc[i:i+target_size])

    return np.array(data), np.array(labels)


In [32]:
x=dataset[future]
y=dataset['NH3']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,shuffle=False,random_state=25)
train_dataset,train_labels=multivariate_data(x_train,y_train,0,2800,3,1,1,False)
test_dataset,test_labels=multivariate_data(x_test,y_test,0,1200,3,1,1,False)

In [ ]:
def create_batch_dataset(x,y,train=True,buffer_size=600,batch_size=32):
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

train_batch_dataset=create_batch_dataset(train_dataset,train_labels)
test_batch_dataset=create_batch_dataset(test_dataset,test_labels,train=False)


model= tf.keras.models.Sequential([
    tf.keras.layers.LSTM(256, input_shape=train_dataset.shape[-2:],return_sequences=True), 
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam',loss='mae')

model.summary()


dot_graph = model_to_dot(model, show_shapes=True)


with open('3层单向LSTM.png', 'wb') as f:
    f.write(dot_graph.create_png())

early_stop = EarlyStopping(monitor='val_loss', patience=15)

history=model.fit(train_batch_dataset,epochs=128,validation_data=test_batch_dataset,callbacks=[early_stop])

In [ ]:

train_predictions = model.predict(train_dataset)
test_predictions = model.predict(test_dataset)


train_predictions = scaler.inverse_transform(train_predictions)
train_labels = scaler.inverse_transform(train_labels)
test_predictions = scaler.inverse_transform(test_predictions)
test_labels = scaler.inverse_transform(test_labels)

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(train_labels, label='train true')
plt.plot(train_predictions, label='train pred')
plt.legend()
plt.show()
plt.figure(figsize=(10, 6))
plt.plot(test_labels, label='test true')
plt.plot(test_predictions, label='test pred')
plt.legend()
plt.show()

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
train_mse = mean_squared_error(train_labels, train_predictions)
train_rmse=np.sqrt(metrics.mean_squared_error(train_labels, train_predictions))
train_mae = mean_absolute_error(train_labels, train_predictions)
train_r2 = r2_score(train_labels, train_predictions)
def train_mape(train_labels, train_predictions):
    return np.mean(np.abs((train_labels, train_predictions)/train_predictions))              
train_mape =train_mape(train_labels, train_predictions)
print('train_mse:',train_mse)
print('train_rmse:',train_rmse)
print('train_mae:',train_mae)
print('train_mape:',train_mape)
print('train_r2:',train_r2)
test_mse = mean_squared_error(test_labels, test_predictions)
test_rmse=np.sqrt(metrics.mean_squared_error(test_labels, test_predictions))
test_mae = mean_absolute_error(test_labels, test_predictions)
test_r2 = r2_score(test_labels, test_predictions)
def test_mape(test_labels, test_predictions):
    return np.mean(np.abs((test_labels, test_predictions)/test_predictions))              
test_mape =test_mape(test_labels, test_predictions)
print('test_mse:',test_mse)
print('test_rmse:',test_rmse)
print('test_mae:',test_mae)
print('test_mape:',test_mape)
print('test_r2:',test_r2)